In [108]:
# Install required libraries
!pip install numpy
!pip install transformers
!pip install datasets
!pip install peft
!pip install ipywidgets --upgrade
!pip install scikit-learn

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: pip install --upgrade pip


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: pip install --upgrade pip


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


...


[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: pip install --upgrade pip


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: pip install --upgrade pip


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [109]:
import numpy as np
from transformers import AutoModelForSequenceClassification, AutoTokenizer, Trainer, TrainingArguments, EvalPrediction, DataCollatorWithPadding
from datasets import load_dataset
from peft import LoraConfig, get_peft_model, AutoPeftModelForSequenceClassification
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

In [110]:
# load the model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token
model = AutoModelForSequenceClassification.from_pretrained("gpt2", num_labels=4)
model.config.pad_token_id = tokenizer.pad_token_id

# load the dataset
dataset = load_dataset("ag_news")

# From class example
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=128)

# tokenize the dataset
tokenized_datasets = dataset.map(tokenize_function, batched=True)

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [111]:
def compute_metrics(p: EvalPrediction):
    preds = np.argmax(p.predictions, axis=1)
    precision, recall, f1, _ = precision_recall_fscore_support(p.label_ids, preds, average='macro', zero_division=1)
    acc = accuracy_score(p.label_ids, preds)
    return {"accuracy": acc, "f1": f1, "precision": precision, "recall": recall}

In [112]:
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=1,
)

# made smaller subset for sake of running in time
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"].shuffle().select([i for i in range(100)]),
    eval_dataset=tokenized_datasets["test"].shuffle().select([i for i in range(100)]),
    compute_metrics=compute_metrics
)

In [113]:
# evaluate initial model without training
initial_results = trainer.evaluate()
print(f"Initial results: {initial_results}")

Initial results: {'eval_loss': 5.065890789031982, 'eval_model_preparation_time': 0.002, 'eval_accuracy': 0.21, 'eval_f1': 0.09972041006523766, 'eval_precision': 0.5827228327228327, 'eval_recall': 0.20892857142857144, 'eval_runtime': 3.2466, 'eval_samples_per_second': 30.802, 'eval_steps_per_second': 4.004}


In [114]:
# LoRA configuration
config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["attn.c_attn", "mlp.c_fc", "mlp.c_proj"],
    lora_dropout=0.1,
    bias="none",
    fan_in_fan_out = True
)

# create a PEFT model
lora_model = get_peft_model(model, config)
lora_model.print_trainable_parameters()

# train the model now
trainer.model = lora_model
trainer.train()
lora_model.save_pretrained("peft-lora")


trainable params: 2,064,384 || all params: 126,507,264 || trainable%: 1.6318


Epoch,Training Loss,Validation Loss,Model Preparation Time,Accuracy,F1,Precision,Recall
1,No log,4.924701,0.004200,0.180000,0.089345,0.566220,0.178929


In [115]:
# load the model
tokenizer = AutoTokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token
lora_model = AutoPeftModelForSequenceClassification.from_pretrained("peft-lora")
lora_model.config.pad_token_id = tokenizer.pad_token_id
trainer = Trainer(
    model=lora_model,
    args=training_args,
    eval_dataset=tokenized_datasets["test"].shuffle().select([i for i in range(100)]),
    compute_metrics=compute_metrics
)


# now evaluate trained model
trainer.model = lora_model
fine_tuned_result = trainer.evaluate()
print(f"Fine-tuned results: {fine_tuned_result}")

# use accuracy to evaluate improvement
initial_accuracy = initial_results["eval_accuracy"]
fine_tuned_accuracy = fine_tuned_result["eval_accuracy"]
change = fine_tuned_accuracy - initial_accuracy
print(f"Accuracy Improvement: {change}")

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Fine-tuned results: {'eval_loss': 1.9648971557617188, 'eval_model_preparation_time': 0.0043, 'eval_accuracy': 0.22, 'eval_f1': 0.09016393442622951, 'eval_precision': 0.8049999999999999, 'eval_recall': 0.25, 'eval_runtime': 3.7354, 'eval_samples_per_second': 26.771, 'eval_steps_per_second': 3.48}
Accuracy Improvement: 0.010000000000000009
